In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import logging
from sklearn.neighbors import NearestNeighbors
import os

# Configuración de logging
logging.basicConfig(
  filename='procesamiento_datos.log',
  level=logging.INFO,
  format='%(asctime)s - %(levelname)s - %(message)s'
)

In [18]:
# Lectura de archivos Excel
try:
    estacionados_camion = pd.read_excel('../Limpia/estacionados_camion.xlsx')
    df_tareas = pd.read_excel('../Limpia/Tareas-limpio.xlsx')
    ubi_cliente = pd.read_excel('../Limpia/Ubicaciones_direcciones.xlsx')
    print("Archivos Excel leídos correctamente.")
except Exception as e:
    print(f"Error al leer los archivos Excel: {e}")
    exit()

Archivos Excel leídos correctamente.


In [19]:
print("______ INFO DF_TAREAS ______ ")
print("           ")
df_tareas.info()
print("           ")
print("______ INFO UBI_CLIENTE ______ ")
print("           ")
ubi_cliente.info()
print("           ")
print("______ INFO ESTACIONADOS_CAMION ______ ")
print("           ")
estacionados_camion.info()

______ INFO DF_TAREAS ______ 
           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1841 entries, 0 to 1840
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   CODIGO    1841 non-null   int64 
 1   PROYECTO  1841 non-null   object
dtypes: int64(1), object(1)
memory usage: 28.9+ KB
           
______ INFO UBI_CLIENTE ______ 
           
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     730 non-null    int64         
 1   NOMCLI     729 non-null    object        
 2   LATITUD    730 non-null    float64       
 3   UBICACIÓN  730 non-null    object        
 4   LONGITUD   730 non-null    float64       
 5   FECHA      730 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 34.3+ KB
           
______ INFO ESTACIO

In [20]:
estacionados_camion.tail(5)

,Indice,Numero_de_placa,Estado_de_viaje,Tiempo_de_Inicio,Tiempo_Final,Duracion,Lugar_de_inicio,camion_x,camion_y
1956,724,PartnerABG9758,Estacionamiento,2025-07-25 10:02:19,2025-07-25 10:51:53,0.0,"34.702330S,55.886721W",-34.702330,-55.886721
1957,726,PartnerABG9758,Estacionamiento,2025-07-25 10:53:36,2025-07-25 11:17:39,0.0,"34.717143S,55.878973W",-34.717143,-55.878973
1958,728,PartnerABG9758,Estacionamiento,2025-07-25 11:18:47,2025-07-25 11:34:22,0.0,"34.737754S,55.873833W",-34.737754,-55.873833
1959,730,PartnerABG9758,Estacionamiento,2025-07-25 12:01:20,2025-07-25 12:05:16,0.0,"34.772738S,55.763545W",-34.772738,-55.763545
1960,732,PartnerABG9758,Estacionamiento,2025-07-25 12:06:48,2025-07-25 12:09:23,0.0,"34.774658S,55.762470W",-34.774658,-55.762470


## === INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===
### Asignación de Cliente más Cercano

In [21]:
print("\n=== INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===")

print("\n1. PREPARANDO DATOS PARA EL MODELO")
print("-----------------------------------")
# Usamos directamente los DataFrames originales
X = ubi_cliente[['LATITUD', 'LONGITUD']].values
y = estacionados_camion[['camion_x', 'camion_y']].values
print(f"Matriz de coordenadas de clientes (X): {X.shape}")
print(f"Matriz de coordenadas de camiones (y): {y.shape}")

if X.shape[0] == 0 or y.shape[0] == 0:
    print("❌ No hay suficientes datos con coordenadas válidas")
else:
    print("\n2. ENTRENANDO MODELO NEAREST NEIGHBORS")
    print("--------------------------------------")
    print("Configuración del modelo:")
    print("- n_neighbors: 1")
    print("- algorithm: auto")
    print("- metric: euclidean")
    print("- p: 2 (norma L2)")
    
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='euclidean', p=2).fit(X)
    print("✓ Modelo entrenado exitosamente")




=== INICIO DE ASIGNACIÓN DE CLIENTES A CAMIONES ===

1. PREPARANDO DATOS PARA EL MODELO
-----------------------------------
Matriz de coordenadas de clientes (X): (730, 2)
Matriz de coordenadas de camiones (y): (1961, 2)

2. ENTRENANDO MODELO NEAREST NEIGHBORS
--------------------------------------
Configuración del modelo:
- n_neighbors: 1
- algorithm: auto
- metric: euclidean
- p: 2 (norma L2)
✓ Modelo entrenado exitosamente


In [22]:

    print("\n3. CALCULANDO DISTANCIAS Y ASIGNANDO CLIENTES")
    print("--------------------------------------------")
    distances, indices = nbrs.kneighbors(y)
    
    # Estadísticas de las distancias
    print("\nEstadísticas de distancias (en grados):")
    print(f"- Distancia mínima: {distances.min():.4f}°")
    print(f"- Distancia máxima: {distances.max():.4f}°")
    print(f"- Distancia promedio: {distances.mean():.4f}°")
    print(f"- Desviación estándar: {distances.std():.4f}°")
    
    # Distribución de distancias
    print("\nDistribución de distancias:")
    percentiles = [25, 50, 75]
    for p in percentiles:
        print(f"- Percentil {p}: {np.percentile(distances, p):.4f}°")
    
    # Conteo de asignaciones
    print(f"\nTotal de asignaciones realizadas: {len(indices)}")
    
    # Análisis de clientes asignados: usamos ubi_cliente directamente
    clientes_asignados = ubi_cliente.iloc[indices.flatten()]['CODIGO'].value_counts()
    print("\nEstadísticas de asignación de clientes:")
    print(f"- Clientes únicos asignados: {len(clientes_asignados)}")
    if len(clientes_asignados) > 0:
        print(f"- Máximo de veces que se asignó un mismo cliente: {clientes_asignados.max()}")
        print("\nTop 5 clientes más asignados:")
        for codigo, count in clientes_asignados.head().items():
            print(f"  Cliente {codigo}: {count} veces")


3. CALCULANDO DISTANCIAS Y ASIGNANDO CLIENTES
--------------------------------------------

Estadísticas de distancias (en grados):
- Distancia mínima: 0.0000°
- Distancia máxima: 0.1599°
- Distancia promedio: 0.0012°
- Desviación estándar: 0.0081°

Distribución de distancias:
- Percentil 25: 0.0001°
- Percentil 50: 0.0002°
- Percentil 75: 0.0006°

Total de asignaciones realizadas: 1961

Estadísticas de asignación de clientes:
- Clientes únicos asignados: 336
- Máximo de veces que se asignó un mismo cliente: 144

Top 5 clientes más asignados:
  Cliente 80283: 144 veces
  Cliente 11168: 37 veces
  Cliente 11975: 24 veces
  Cliente 12951: 24 veces
  Cliente 62867: 24 veces


In [23]:


    print("\n4. ASIGNANDO CÓDIGOS DE CLIENTES A CAMIONES")
    print("------------------------------------------")
    # Asignamos directamente sobre estacionados_camion
    estacionados_camion['CODIGO'] = ubi_cliente.iloc[indices.flatten()]['CODIGO'].values
    print("✓ Códigos asignados exitosamente")
    
    # Añadimos las distancias al DataFrame
    estacionados_camion['distancia_al_cliente'] = distances.flatten()
    print("\nEstadísticas de asignaciones:")
    print(f"- Asignaciones a menos de 1°: {(distances < 1).sum()} ({(distances < 1).sum()/len(distances)*100:.1f}%)")
    print(f"- Asignaciones a menos de 5°: {(distances < 5).sum()} ({(distances < 5).sum()/len(distances)*100:.1f}%)")
    print(f"- Asignaciones a menos de 10°: {(distances < 10).sum()} ({(distances < 10).sum()/len(distances)*100:.1f}%)")

    print("\n5. RESULTADOS FINALES")
    print("--------------------------------")
    # Como no hay filtrado, usamos directamente estacionados_camion como resultado final
    estacionados_camion_final = estacionados_camion.copy()
    total_camiones = len(estacionados_camion_final)
    camiones_asignados = estacionados_camion_final['CODIGO'].notnull().sum()
    camiones_sin_asignacion = total_camiones - camiones_asignados
    
    print(f"Total de registros en resultado final: {total_camiones}")
    print(f"- Camiones con cliente asignado: {camiones_asignados}")
    print(f"- Camiones sin asignación: {camiones_sin_asignacion}")
    
    # Resumen final de calidad de asignaciones
    print("\nResumen de calidad de asignaciones:")
    print(f"- Porcentaje de camiones con asignación: {camiones_asignados/total_camiones*100:.1f}%")
    print(f"- Distancia promedio a clientes: {distances.mean():.4f}°")
    print(f"- Mediana de distancia a clientes: {np.median(distances):.4f}°")

print("\n=== FIN DE ASIGNACIÓN DE CLIENTES A CAMIONES ===")


4. ASIGNANDO CÓDIGOS DE CLIENTES A CAMIONES
------------------------------------------
✓ Códigos asignados exitosamente

Estadísticas de asignaciones:
- Asignaciones a menos de 1°: 1961 (100.0%)
- Asignaciones a menos de 5°: 1961 (100.0%)
- Asignaciones a menos de 10°: 1961 (100.0%)

5. RESULTADOS FINALES
--------------------------------
Total de registros en resultado final: 1961
- Camiones con cliente asignado: 1961
- Camiones sin asignación: 0

Resumen de calidad de asignaciones:
- Porcentaje de camiones con asignación: 100.0%
- Distancia promedio a clientes: 0.0012°
- Mediana de distancia a clientes: 0.0002°

=== FIN DE ASIGNACIÓN DE CLIENTES A CAMIONES ===


In [24]:
estacionados_camion_final.sample(5)

,Indice,Numero_de_placa,Estado_de_viaje,Tiempo_de_Inicio,Tiempo_Final,Duracion,Lugar_de_inicio,camion_x,camion_y,CODIGO,distancia_al_cliente
772,228,CAT 1006 - 4G,Estacionamiento,2025-07-05 10:50:33,2025-07-05 11:27:24,0.0,"34.778142S,55.857087W",-34.778142,-55.857087,12916,0.000076
1956,724,PartnerABG9758,Estacionamiento,2025-07-25 10:02:19,2025-07-25 10:51:53,0.0,"34.702330S,55.886721W",-34.702330,-55.886721,42782,0.000509
160,402,PartnerABG9758,Estacionamiento,2025-05-18 16:53:48,2025-05-18 17:07:23,0.0,"34.762294S,55.746326W",-34.762294,-55.746326,81176,0.000793
1237,132,Peugeot AVG9758 - 4G,Estacionamiento,2025-07-04 08:43:51,2025-07-04 08:49:55,0.0,"34.756573S,55.724327W",-34.756573,-55.724327,11998,0.000309
1571,866,Peugeot AVG9758 - 4G,Estacionamiento,2025-07-22 08:40:06,2025-07-22 08:53:59,0.0,"34.754310S,55.675703W",-34.754310,-55.675703,22644,0.000075


## === VERIFICACION DE DF TAREAS CODIGO  ===

# Procesamiento de df_tareas
print("\n1. PROCESANDO DF_TAREAS")
print("----------------------")
print("Verificando y limpiando códigos...")

# Identificar registros inválidos
df_tareas_invalidas = df_tareas[
  df_tareas['CODIGO'].isnull() | 
  (df_tareas['CODIGO'].apply(lambda x: not str(x).isdigit()))
].copy()
print(f"Filas con 'CODIGO' nulo o no numérico: {df_tareas_invalidas.shape[0]}")

# Limpiar y convertir CODIGO
df_tareas = df_tareas.dropna(subset=['CODIGO']).copy()
df_tareas['CODIGO'] = pd.to_numeric(df_tareas['CODIGO'], errors='coerce')
filas_iniciales = df_tareas.shape[0]
df_tareas = df_tareas.dropna(subset=['CODIGO']).copy()
filas_finales = df_tareas.shape[0]
df_tareas['CODIGO'] = df_tareas['CODIGO'].astype(int)

print(f"Filas eliminadas: {filas_iniciales - filas_finales}")
print(f"Filas restantes: {filas_finales}")

# Filtrar columnas necesarias
df_tareas_filtrado = df_tareas[['CODIGO', 'PROYECTO']].copy()
print(f"Columnas seleccionadas: {list(df_tareas_filtrado.columns)}")


In [25]:
print("\n2. PROCESANDO ESTACIONADOS_CAMION")
print("--------------------------------")
# Convertir columnas de tiempo
for columna in ['Tiempo_de_Inicio', 'Tiempo_Final']:
  if columna in estacionados_camion.columns:
      estacionados_camion[columna] = pd.to_datetime(estacionados_camion[columna], errors='coerce')
      print(f"✓ Columna '{columna}' convertida a datetime")

# Filtrar camiones estacionados
if 'Estado_de_viaje' in estacionados_camion.columns:
  total_camiones = estacionados_camion.shape[0]
  estacionados_camion_filtered = estacionados_camion[
      estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'
  ].copy()
  print(f"Camiones estacionados: {estacionados_camion_filtered.shape[0]} de {total_camiones}")
else:
  print("❌ Error: Falta columna 'Estado_de_viaje'")
  raise ValueError("Columna 'Estado_de_viaje' no encontrada")

print("\n=== FIN DE LIMPIEZA DE DATOS ===")


2. PROCESANDO ESTACIONADOS_CAMION
--------------------------------
✓ Columna 'Tiempo_de_Inicio' convertida a datetime
✓ Columna 'Tiempo_Final' convertida a datetime
Camiones estacionados: 1961 de 1961

=== FIN DE LIMPIEZA DE DATOS ===


In [26]:

print("\n2. PROCESANDO ESTACIONADOS_CAMION")
print("--------------------------------")
# Convertir columnas de tiempo
for columna in ['Tiempo_de_Inicio', 'Tiempo_Final']:
  if columna in estacionados_camion.columns:
      estacionados_camion[columna] = pd.to_datetime(estacionados_camion[columna], errors='coerce')
      print(f"✓ Columna '{columna}' convertida a datetime")

# Filtrar camiones estacionados
if 'Estado_de_viaje' in estacionados_camion.columns:
  total_camiones = estacionados_camion.shape[0]
  estacionados_camion_filtered = estacionados_camion[
      estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'
  ].copy()
  print(f"Camiones estacionados: {estacionados_camion_filtered.shape[0]} de {total_camiones}")
else:
  print("❌ Error: Falta columna 'Estado_de_viaje'")
  raise ValueError("Columna 'Estado_de_viaje' no encontrada")

print("\n=== FIN DE LIMPIEZA DE DATOS ===")


2. PROCESANDO ESTACIONADOS_CAMION
--------------------------------
✓ Columna 'Tiempo_de_Inicio' convertida a datetime
✓ Columna 'Tiempo_Final' convertida a datetime
Camiones estacionados: 1961 de 1961

=== FIN DE LIMPIEZA DE DATOS ===


In [27]:
estacionados_camion_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1961 entries, 0 to 1960
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Indice                1961 non-null   int64         
 1   Numero_de_placa       1961 non-null   object        
 2   Estado_de_viaje       1961 non-null   object        
 3   Tiempo_de_Inicio      1961 non-null   datetime64[ns]
 4   Tiempo_Final          1961 non-null   datetime64[ns]
 5   Duracion              1961 non-null   float64       
 6   Lugar_de_inicio       1961 non-null   object        
 7   camion_x              1961 non-null   float64       
 8   camion_y              1961 non-null   float64       
 9   CODIGO                1961 non-null   int64         
 10  distancia_al_cliente  1961 non-null   float64       
dtypes: datetime64[ns](2), float64(4), int64(2), object(3)
memory usage: 168.6+ KB


## === INICIO DE PROCESO DE MERGES ===

Este merge es el que tenemos que corregir!


print("\n=== INICIO DE PROCESO DE MERGES ===")

print("\n1. PRIMER MERGE: df_tareas_filtrado con ubi_cliente")
print("---------------------------------------------")
print(f"Estado inicial:")
print(f"- df_tareas_filtrado: {df_tareas_filtrado.shape} filas")
print(f"- ubi_cliente: {ubi_cliente.shape} filas")

# Primer merge
merged_df = pd.merge(df_tareas_filtrado, ubi_cliente, on='CODIGO', how='inner')
print("\nResultado del primer merge:")
print(f"- Filas resultantes: {merged_df.shape[0]}")
print(f"- Columnas resultantes: {merged_df.shape[1]}")
print(f"- Columnas: {merged_df.columns.tolist()}")


### Prueba merge outer para detectar problemas

# Hacemos un merge con how='outer' para ver qué registros se pierden en cada DataFrame
merged_df_test = pd.merge(df_tareas_filtrado, ubi_cliente, on='CODIGO', how='outer', indicator=True)

# Contamos cuántos registros vienen de cada DataFrame
print("\n=== Resultados del merge con OUTER ===")
print(merged_df_test['_merge'].value_counts())

# Filtramos los registros que vienen solo de cada tabla
solo_en_tareas = merged_df_test[merged_df_test['_merge'] == 'left_only']
solo_en_clientes = merged_df_test[merged_df_test['_merge'] == 'right_only']

# Mostramos cuántos registros están en una tabla pero no en la otra
print(f"\nRegistros en df_tareas_filtrado que NO tienen match en ubi_cliente: {len(solo_en_tareas)}")
print(f"Registros en ubi_cliente que NO tienen match en df_tareas_filtrado: {len(solo_en_clientes)}")

# Opcional: Guardar los registros que no tienen match para analizarlos
solo_en_tareas.to_excel("solo_en_tareas.xlsx", index=False)
solo_en_clientes.to_excel("solo_en_clientes.xlsx", index=False)

In [28]:
ubi_cliente.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   CODIGO     730 non-null    int64         
 1   NOMCLI     729 non-null    object        
 2   LATITUD    730 non-null    float64       
 3   UBICACIÓN  730 non-null    object        
 4   LONGITUD   730 non-null    float64       
 5   FECHA      730 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 34.3+ KB


In [29]:


# Usamos estacionados_camion_final que ya tiene la columna CODIGO asignada
merged_final_df = pd.merge(
  ubi_cliente, 
  estacionados_camion_final,  # Usamos el DataFrame que ya tiene CODIGO
  on='CODIGO', 
  how='inner',
  suffixes=('_clientes', '_camion')
)
print(f"\nResultado del merge final:")
print(f"- Filas: {merged_final_df.shape[0]}")
print(f"- Columnas: {merged_final_df.shape[1]}")

print("\n3. SELECCIÓN DE COLUMNAS FINALES")
print("------------------------------")

columnas_finales = [
  'CODIGO', 'NOMCLI', 'Duracion', 'Tiempo_de_Inicio', 'UBICACIÓN',
  'Numero_de_placa', 'LATITUD', 'LONGITUD', 'camion_x', 'camion_y'
]



merged_final_df = merged_final_df[columnas_finales]



Resultado del merge final:
- Filas: 1967
- Columnas: 16

3. SELECCIÓN DE COLUMNAS FINALES
------------------------------


In [30]:
merged_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967 entries, 0 to 1966
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CODIGO            1967 non-null   int64         
 1   NOMCLI            1961 non-null   object        
 2   Duracion          1967 non-null   float64       
 3   Tiempo_de_Inicio  1967 non-null   datetime64[ns]
 4   UBICACIÓN         1967 non-null   object        
 5   Numero_de_placa   1967 non-null   object        
 6   LATITUD           1967 non-null   float64       
 7   LONGITUD          1967 non-null   float64       
 8   camion_x          1967 non-null   float64       
 9   camion_y          1967 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 153.8+ KB


print("\n2. MERGE FINAL")
print("------------")
print("Realizando merge final entre merged_df y estacionados_camion_final")
# Usamos estacionados_camion_final que ya tiene la columna CODIGO asignada
merged_final_df = pd.merge(
  merged_df, 
  estacionados_camion_final,  # Usamos el DataFrame que ya tiene CODIGO
  on='CODIGO', 
  how='inner',
  suffixes=('_tareas', '_camion')
)
print(f"\nResultado del merge final:")
print(f"- Filas: {merged_final_df.shape[0]}")
print(f"- Columnas: {merged_final_df.shape[1]}")

print("\n3. SELECCIÓN DE COLUMNAS FINALES")
print("------------------------------")




# Verificar columnas disponibles
columnas_faltantes = [col for col in columnas_finales if col not in merged_final_df.columns]
if columnas_faltantes:
  print("\n⚠️ Columnas faltantes:")
  for col in columnas_faltantes:
      print(f"- {col}")
else:
  filtered_df = merged_final_df[columnas_finales].copy()
  print("\nColumnas seleccionadas exitosamente:")
  for col in columnas_finales:
      print(f"- {col}")

In [31]:
merged_final_df [merged_final_df ['NOMCLI'].isnull()]
merged_final_df .loc[
    (merged_final_df ['CODIGO'] == 81121) & (merged_final_df ['NOMCLI'].isnull()),
    'NOMCLI'
] = 'DEREK GONZALEZ'

## GUARDANDO RESULTADO

In [32]:

print("\n4. GUARDANDO RESULTADO")
print("--------------------")
try:
  merged_final_df .to_excel('../Limpia/merged_df.xlsx', index=False)
  print("✓ Archivo guardado exitosamente")
  
  print("\nVerificación final de nulos:")
  nulos = merged_final_df .isnull().sum()
  for columna, cantidad in nulos.items():
      print(f"- {columna}: {cantidad} nulos")
except Exception as e:
  print(f"❌ Error al guardar: {e}")

print("\n=== FIN DE PROCESO DE MERGES ===")


4. GUARDANDO RESULTADO
--------------------
✓ Archivo guardado exitosamente

Verificación final de nulos:
- CODIGO: 0 nulos
- NOMCLI: 0 nulos
- Duracion: 0 nulos
- Tiempo_de_Inicio: 0 nulos
- UBICACIÓN: 0 nulos
- Numero_de_placa: 0 nulos
- LATITUD: 0 nulos
- LONGITUD: 0 nulos
- camion_x: 0 nulos
- camion_y: 0 nulos

=== FIN DE PROCESO DE MERGES ===


In [33]:
merged_final_df .info()
merged_final_df .tail(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1967 entries, 0 to 1966
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   CODIGO            1967 non-null   int64         
 1   NOMCLI            1967 non-null   object        
 2   Duracion          1967 non-null   float64       
 3   Tiempo_de_Inicio  1967 non-null   datetime64[ns]
 4   UBICACIÓN         1967 non-null   object        
 5   Numero_de_placa   1967 non-null   object        
 6   LATITUD           1967 non-null   float64       
 7   LONGITUD          1967 non-null   float64       
 8   camion_x          1967 non-null   float64       
 9   camion_y          1967 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(3)
memory usage: 153.8+ KB


,CODIGO,NOMCLI,Duracion,Tiempo_de_Inicio,UBICACIÓN,Numero_de_placa,LATITUD,LONGITUD,camion_x,camion_y
1962,13090,DANIEL MERCADO SALINAS,0.0,2025-07-10 15:38:14,"http://maps.google.com/?q=-34.78022618534145,-...",CAT 1006 - 4G,-34.780226,-55.838609,-34.779494,-55.839149
1963,13090,DANIEL MERCADO SALINAS,0.0,2025-07-12 14:17:41,"http://maps.google.com/?q=-34.78022618534145,-...",CAT 1006 - 4G,-34.780226,-55.838609,-34.779971,-55.839074
1964,13090,DANIEL MERCADO SALINAS,0.0,2025-07-24 15:25:48,"http://maps.google.com/?q=-34.78022618534145,-...",CAT 1006 - 4G,-34.780226,-55.838609,-34.779943,-55.839115
1965,13090,DANIEL MERCADO SALINAS,0.0,2025-07-24 15:35:04,"http://maps.google.com/?q=-34.78022618534145,-...",CAT 1006 - 4G,-34.780226,-55.838609,-34.780264,-55.838619
1966,13090,DANIEL MERCADO SALINAS,0.0,2025-07-15 14:13:35,"http://maps.google.com/?q=-34.78022618534145,-...",PartnerABG9758,-34.780226,-55.838609,-34.779493,-55.839164
